# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [84]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [85]:
# 2) seu código aqui
sinasc.isna().sum().sort_values(ascending=False)



DTRECORIGA    27028
CODANOMAL     26814
IDADEPAI      19421
SERIESCMAE    12710
DTULTMENST    10072
              ...  
PESO              0
DTCADASTRO        0
NUMEROLOTE        0
VERSAOSIST        0
ORIGEM            0
Length: 69, dtype: int64

In [112]:
# 3) seu código aqui
sinasc = sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
sinasc.isna().sum().sort_values(ascending=False)

QTDFILVIVO    1573
GESTACAO      1232
ESTCIVMAE      317
ESCMAE         312
APGAR5         103
GRAVIDEZ        79
LOCNASC          0
IDADEMAE         0
CONSULTAS        0
dtype: int64

In [123]:
# 4) seu código aqui
sinasc_valid_apgar5 = sinasc[sinasc['APGAR5'].notna()]
print(f'Novo df possuí {sinasc_valid_apgar5.shape[0]} linhas, antes possuia {sinasc.shape[0]}.')
sinasc_valid_apgar5[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']].isna().sum().sort_values(ascending=False)

Novo df possuí 26925 linhas, antes possuia 27028.


QTDFILVIVO    1566
GESTACAO      1216
ESTCIVMAE      315
ESCMAE         310
GRAVIDEZ        76
LOCNASC          0
IDADEMAE         0
CONSULTAS        0
APGAR5           0
dtype: int64

In [126]:
# 5) seu código aqui
sinasc_valid_apgar5['CONSULTAS'].fillna(9, inplace=True)
sinasc_valid_apgar5['ESTCIVMAE'].fillna(9, inplace=True)


/tmp/ipykernel_1396052/4204194880.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_valid_apgar5['CONSULTAS'].fillna(9, inplace=True)
/tmp/ipykernel_1396052/4204194880.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_valid_apgar5['ESTCIVMAE'].fillna(9, inplace=True)


QTDFILVIVO    1566
GESTACAO      1216
ESCMAE         310
GRAVIDEZ        76
LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
CONSULTAS        0
APGAR5           0
dtype: int64

In [128]:
# 6) Seu código aqui
sinasc_valid_apgar5['QTDFILVIVO'].fillna(0, inplace=True)
sinasc_valid_apgar5.isna().sum().sort_values(ascending=False)


/tmp/ipykernel_1396052/3186620828.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_valid_apgar5['QTDFILVIVO'].fillna(0, inplace=True)


GESTACAO      1216
ESCMAE         310
GRAVIDEZ        76
LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
QTDFILVIVO       0
CONSULTAS        0
APGAR5           0
dtype: int64

In [143]:
# 7) seu código aqui
gravidez = sinasc_valid_apgar5['GRAVIDEZ'].mode()[0]
gestacao = sinasc_valid_apgar5['GESTACAO'].mode()[0]
escmae = sinasc_valid_apgar5['ESCMAE'].mode()[0]
sinasc_valid_apgar5['GRAVIDEZ'].fillna(gravidez, inplace=True)
sinasc_valid_apgar5['GESTACAO'].fillna(gestacao, inplace=True)
sinasc_valid_apgar5['ESCMAE'].fillna(escmae, inplace=True)


## Valores serão substituídos pela moda, pois são valores categóricos e esse valor altera menos a distribuição dos dados.


4727.0

In [148]:
# 8) seu código aqui
## - Entre 8 e 10 está em uma faixa 'normal'. 
## - Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
## - Entre 4 e 5 significa 'asfixia moderada'.
## - Entre 0 e 3 significa 'asfixia severa'. 

sinasc_valid_apgar5.loc[sinasc_valid_apgar5['APGAR5'] < 4, 'nivel_asfixia'] = 'asfixia severa'
sinasc_valid_apgar5.loc[(sinasc_valid_apgar5['APGAR5'] >= 4) & (sinasc_valid_apgar5['APGAR5'] <= 5), 'nivel_asfixia'] = 'asfixia moderada'
sinasc_valid_apgar5.loc[(sinasc_valid_apgar5['APGAR5'] >= 6) & (sinasc_valid_apgar5['APGAR5'] <= 7), 'nivel_asfixia'] = 'asfixia leve'
sinasc_valid_apgar5.loc[sinasc_valid_apgar5['APGAR5'] >= 8, 'nivel_asfixia'] = 'normal'
sinasc_valid_apgar5['nivel_asfixia'].value_counts()

/tmp/ipykernel_1396052/2230571589.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_valid_apgar5.loc[sinasc_valid_apgar5['APGAR5'] < 4, 'nivel_asfixia'] = 'asfixia severa'


normal              26463
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: nivel_asfixia, dtype: int64

In [152]:
# 9) seu código aqui
sinasc_valid_apgar5.rename(
    columns= {
        'LOCNASC': 'local_nascimento',
        'IDADEMAE': 'idade_mae',
        'ESTCIVMAE': 'estado_civil_mae',
        'ESCMAE': 'escolaridade_mae',
        'QTDFILVIVO': 'qtd_filhos_vivos',
        'GESTACAO': 'gestacao',
        'GRAVIDEZ': 'gravidez',
        'CONSULTAS': 'consultas',
        'APGAR5': 'apgar5'
        },
    inplace=True)
sinasc_valid_apgar5.columns

/tmp/ipykernel_1396052/703069039.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_valid_apgar5.rename(


Index(['local_nascimento', 'idade_mae', 'estado_civil_mae', 'escolaridade_mae',
       'qtd_filhos_vivos', 'gestacao', 'gravidez', 'consultas', 'apgar5',
       'nivel_asfixia'],
      dtype='object')